# File Change Matrix Visualizer

This notebook demonstrates the FileChangeMatrixVisualizer with proper pandas display configuration.

In [1]:
from pathlib import Path
from tardisbase.testing.regression_comparison.compare import ReferenceComparer
from tardisbase.testing.regression_comparison.run_tests import run_tests
from tardisbase.testing.regression_comparison.visualize_files import (
    FileChangeMatrixVisualizer, 
)

In [2]:
# Configure pandas display options for better visualization
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Configuration for the analysis
config = {
    "tardis_repo_path": "/home/riddhi/workspace/tardis-main/tardis",
    "regression_data_repo_path": "/home/riddhi/workspace/tardis-main/tardis-regression-data",
    "branch": "master",
    "n": 3,
    "target_file": "tardis/spectrum/tests/test_spectrum_solver/test_spectrum_solver/TestSpectrumSolver.h5",
    "commits": ["300e565e83112528faaa76e970057ffb1b13f743", "2a06fdfb60190bbd9b49ff572d78772607138660", "2d775dcd1c486227532f537fc41066e942000e56"],  # Uncomment for specific commits
    # "commits": commits.calculate_commits(n=10, gap=0, info=False), #Uncomment for n commits with custom gap
    "use_conda": True,
    "conda_manager": "conda"
}

## Option 1: Run tests to generate regression data commits

Uncomment and run this cell if you want to generate new regression data commits:

In [4]:
processed_commits, regression_commits, original_head, target_file_path = run_tests(
        config["tardis_repo_path"], 
        config["regression_data_repo_path"], 
        config["branch"], config["target_file"], 
        n=config["n"], 
        use_conda=config["use_conda"],
        conda_manager=config["conda_manager"]
    )

Original HEAD of regression data repo: 91671169db6f0390dedc659e8e08e6a3823cbdd0
Processing commit 1/3: e0cb6779fcf39d85d029dca7ac7bd1acdf771357
Creating conda environment: tardis-test-e0cb6779
Checking if environment tardis-test-e0cb6779 exists...
Environment tardis-test-e0cb6779 already exists, skipping creation.
Installing TARDIS in environment: conda run -n tardis-test-e0cb6779 pip install -e /home/riddhi/workspace/tardis-main/tardis
Running pytest command: conda run -n tardis-test-e0cb6779 python -m pytest tardis/spectrum/tests/test_spectrum_solver.py --tardis-regression-data=/home/riddhi/workspace/tardis-main/tardis-regression-data --generate-reference -x --disable-warnings
Pytest stdout:
============================= test session starts ==============================
platform linux -- Python 3.13.2, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/riddhi/workspace/tardis-main/tardis
configfile: pyproject.toml
plugins: doctestplus-1.4.0, html-4.1.1, anyio-4.9.0, metadata-3.1.1, cov-6.0.0

## Option 2: Use existing regression data commits

Set your regression data commit hashes here:

In [ ]:
# Example regression commits - replace with your actual commit hashes
regression_commits = ["cb01559e8915db275bd13047ee8e68c7c5c3b6ce",
                     "cc038d3315a8ae34de6f2c5a3a4058071f949a2b",
                     "91671169db6f0390dedc659e8e08e6a3823cbdd0"]

# Optional: corresponding TARDIS commits (if these regression commits were generated from specific TARDIS commits)
tardis_commits = ["e0cb6779fcf39d85d029dca7ac7bd1acdf771357",
                 "ab4ef440d31737a76c702f0dec628c5e7c19fd3b",
                 "92ba560a72dc1be067c076d285bb8bc3de35f8d7"]

## Visualize File Changes

Create the visualizer and analyze the commits:

In [6]:
# Create the visualizer
# Case 1: Direct regression data commits (no TARDIS commits)
# visualizer = FileChangeMatrixVisualizer(
#     regression_repo_path=config["regression_data_repo_path"],
#     commits=regression_commits
# )

# Case 2: Regression data commits generated from TARDIS commits
visualizer = FileChangeMatrixVisualizer(
    regression_repo_path=config["regression_data_repo_path"],
    commits=regression_commits,
    tardis_commits=processed_commits,
    tardis_repo_path=config["tardis_repo_path"]
)

In [7]:
# Analyze the commits
visualizer.analyze_commits()

Analyzing 3 commits...
Processing commit 1/3: 3346ca01
Processing commit 2/3: 9bd9f68d
Processing commit 3/3: fb6ac289
Found 103 total files across all commits.
Changed files: 1
Unchanged files: 102


In [8]:
# Display the file change matrix with pandas display configuration
with pd.option_context('display.max_colwidth', None, 'display.width', None, 
                      'display.max_columns', None, 'display.max_rows', None):
    visualizer.print_matrix()


COMMIT INFORMATION (3 commits) - Generated Commits from TARDIS:


,Commit #,Regression Hash,Description,Date
0,1,3346ca01,Regression data for Build docs issues fixes (#3075),2025-07-14 18:29
1,2,9bd9f68d,Regression data for Post-release 2025.05.18 (#3084),2025-07-14 18:32
2,3,fb6ac289,Regression data for Fix tardis version fallback (#3086),2025-07-14 18:35



Changed Files Matrix (1 files):


,Files,3346ca,9bd9f6,fb6ac2
0,tardis/spectrum/tests/test_spectrum_solver/test_spectrum_solver/TestSpectrumSolver.h5,•,*,*



Unchanged Files Table (102 files):


,Files,3346ca,9bd9f6,fb6ac2
0,atom_data/chianti_He.h5,•,•,•
1,atom_data/kurucz_atom_chianti_many.h5,•,•,•
2,atom_data/kurucz_cd23_chianti_H_He.h5,•,•,•
3,atom_data/kurucz_cd23_chianti_Si.h5,•,•,•
4,atom_data/new_kurucz_cd23_chianti_H_He.h5,•,•,•
5,atom_data/nlte_atom_data/TestNLTE_He_Ti.h5,•,•,•
6,tardis/opacities/tests/test_tau_sobolev/test_calculate_sobolev_line_opacity.h5,•,•,•
7,tardis/plasma/equilibrium/tests/test_level_populations/test_level_population_solver/test_solve__collisional_rate_solver0-radiative_transitions0__.h5,•,•,•
8,tardis/plasma/equilibrium/tests/test_rate_matrix/test_rate_matrix_solver__collisional_rate_solver0-radiative_transitions0__.h5,•,•,•
9,tardis/plasma/tests/test_complete_plasmas/test_plasma/TestPlasma.h5,•,•,•



Legend:
──────────────────────────────
  • = unchanged (blue)
  * = modified (gold)
  + = added (green)
  - = deleted (red)
  ∅ = not present (grey)


## Understanding the Output

The visualizer shows:

1. **Commit Information**: Details about each analyzed commit
2. **Changed Files Matrix**: Files that changed in any commit with symbols:
   - `•` (blue) = unchanged
   - `*` (gold) = modified  
   - `+` (green) = added
   - `-` (red) = deleted
   - `∅` (grey) = not present
3. **Unchanged Files Table**: Files that remained unchanged across all commits
4. **Legend**: Symbol meanings and colors

The pandas display configuration ensures that:
- Full file paths are shown without truncation
- All columns and rows are displayed
- Commit descriptions are fully visible